In [107]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    appName("SparkDemo"). \
    config('spark.ui.port','0'). \
    config('spark.shuffle.useOldFethProtocol','true'). \
    config('spark.sql.warehouse.dir',f'/user/{username}/warehouse'). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [108]:
customers_schema = """ member_id string, emp_title string, emp_length string,
home_ownership string, annual_inc float, addr_state string, zip_code string, 
country string, grade string, sub_grade string, verification_status string, 
tot_hi_cred_lim float, application_type string, annual_inc_joint float, verification_status_joint string """

In [110]:
customers_raw_df = spark.read \
.format("csv") \
.option("header","true") \
.schema(customers_schema) \
.load("/user/itv009959/lendingclubproject/raw/customers_data_csv")

In [121]:
customers_raw_df

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
707271898dcabc8b2...,Physician Service...,3 years,RENT,40400.0,CO,801xx,USA,A,A2,Not Verified,68759.0,Individual,null,null
8e1ea10aca3c4ad8f...,Operations,10+ years,MORTGAGE,53000.0,AR,720xx,USA,B,B2,Source Verified,63143.0,Individual,null,null
1d6546a2cbc1fd240...,Underwriter,2 years,RENT,65000.0,ME,040xx,USA,B,B4,Not Verified,66695.0,Individual,null,null
d6208beced388988f...,Crome restorer sp...,10+ years,MORTGAGE,60000.0,IL,606xx,USA,C,C1,Not Verified,68900.0,Individual,null,null
b4af936688c28c165...,Program Coordinator,1 year,RENT,38000.0,FL,322xx,USA,A,A5,Not Verified,76877.0,Individual,null,null
2c04e047879ada04e...,Executive Director,10+ years,MORTGAGE,166000.0,IL,601xx,USA,C,C2,Not Verified,217868.0,Individual,null,null
39dfcd293cb7b2c17...,Emergency Managme...,4 years,MORTGAGE,81000.0,TX,761xx,USA,C,C4,Not Verified,293276.0,Individual,null,null
5e6e1f8ad59c71a0b...,Clinical Applicat...,3 years,MORTGAGE,82000.0,CO,801xx,USA,A,A1,Not Verified,393500.0,Individual,null,null
afd3b57e55eb95ed8...,Systems Analyst 3,4 years,OWN,118030.0,MI,482xx,USA,A,A3,Not Verified,82137.0,Individual,null,null
8b5eed45ac53a0238...,Director of Front...,4 years,RENT,62000.0,NY,110xx,USA,A,A5,Not Verified,17400.0,Individual,null,null


In [122]:
customers_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- annual_inc_joint: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)



In [123]:
customers_df_renamed = customers_raw_df.withColumnRenamed("annual_inc","annual_income") \
.withColumnRenamed("addr_state","address_state") \
.withColumnRenamed("zip_code","address_zipcode") \
.withColumnRenamed("country","address_country") \
.withColumnRenamed("tot_hi_cred_lim","total_high_credit_limit") \
.withColumnRenamed("annual_inc_joint","join_annual_income")

In [124]:
customers_df_renamed

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint
707271898dcabc8b2...,Physician Service...,3 years,RENT,40400.0,CO,801xx,USA,A,A2,Not Verified,68759.0,Individual,null,null
8e1ea10aca3c4ad8f...,Operations,10+ years,MORTGAGE,53000.0,AR,720xx,USA,B,B2,Source Verified,63143.0,Individual,null,null
1d6546a2cbc1fd240...,Underwriter,2 years,RENT,65000.0,ME,040xx,USA,B,B4,Not Verified,66695.0,Individual,null,null
d6208beced388988f...,Crome restorer sp...,10+ years,MORTGAGE,60000.0,IL,606xx,USA,C,C1,Not Verified,68900.0,Individual,null,null
b4af936688c28c165...,Program Coordinator,1 year,RENT,38000.0,FL,322xx,USA,A,A5,Not Verified,76877.0,Individual,null,null
2c04e047879ada04e...,Executive Director,10+ years,MORTGAGE,166000.0,IL,601xx,USA,C,C2,Not Verified,217868.0,Individual,null,null
39dfcd293cb7b2c17...,Emergency Managme...,4 years,MORTGAGE,81000.0,TX,761xx,USA,C,C4,Not Verified,293276.0,Individual,null,null
5e6e1f8ad59c71a0b...,Clinical Applicat...,3 years,MORTGAGE,82000.0,CO,801xx,USA,A,A1,Not Verified,393500.0,Individual,null,null
afd3b57e55eb95ed8...,Systems Analyst 3,4 years,OWN,118030.0,MI,482xx,USA,A,A3,Not Verified,82137.0,Individual,null,null
8b5eed45ac53a0238...,Director of Front...,4 years,RENT,62000.0,NY,110xx,USA,A,A5,Not Verified,17400.0,Individual,null,null


In [125]:
from pyspark.sql.functions import current_timestamp

In [126]:
customers_df_ingested = customers_df_renamed.withColumn("ingest_date",current_timestamp())

In [127]:
customers_df_ingested

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
707271898dcabc8b2...,Physician Service...,3 years,RENT,40400.0,CO,801xx,USA,A,A2,Not Verified,68759.0,Individual,null,null,2024-03-12 17:34:...
8e1ea10aca3c4ad8f...,Operations,10+ years,MORTGAGE,53000.0,AR,720xx,USA,B,B2,Source Verified,63143.0,Individual,null,null,2024-03-12 17:34:...
1d6546a2cbc1fd240...,Underwriter,2 years,RENT,65000.0,ME,040xx,USA,B,B4,Not Verified,66695.0,Individual,null,null,2024-03-12 17:34:...
d6208beced388988f...,Crome restorer sp...,10+ years,MORTGAGE,60000.0,IL,606xx,USA,C,C1,Not Verified,68900.0,Individual,null,null,2024-03-12 17:34:...
b4af936688c28c165...,Program Coordinator,1 year,RENT,38000.0,FL,322xx,USA,A,A5,Not Verified,76877.0,Individual,null,null,2024-03-12 17:34:...
2c04e047879ada04e...,Executive Director,10+ years,MORTGAGE,166000.0,IL,601xx,USA,C,C2,Not Verified,217868.0,Individual,null,null,2024-03-12 17:34:...
39dfcd293cb7b2c17...,Emergency Managme...,4 years,MORTGAGE,81000.0,TX,761xx,USA,C,C4,Not Verified,293276.0,Individual,null,null,2024-03-12 17:34:...
5e6e1f8ad59c71a0b...,Clinical Applicat...,3 years,MORTGAGE,82000.0,CO,801xx,USA,A,A1,Not Verified,393500.0,Individual,null,null,2024-03-12 17:34:...
afd3b57e55eb95ed8...,Systems Analyst 3,4 years,OWN,118030.0,MI,482xx,USA,A,A3,Not Verified,82137.0,Individual,null,null,2024-03-12 17:34:...
8b5eed45ac53a0238...,Director of Front...,4 years,RENT,62000.0,NY,110xx,USA,A,A5,Not Verified,17400.0,Individual,null,null,2024-03-12 17:34:...


In [128]:
customers_df_ingested.count()

2260701

In [129]:
customers_distinct = customers_df_ingested.distinct()

In [130]:
customers_distinct.count()

2260638

In [131]:
customers_distinct.createOrReplaceTempView("customers")

In [132]:
spark.sql("select * from customers")

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
28f813f2dbd389d94...,Business Developm...,10+ years,MORTGAGE,67000.0,IL,604xx,USA,C,C1,Source Verified,186967.0,Individual,null,null,2024-03-12 17:34:...
89c95cc1c0a11f760...,Supervisor,10+ years,RENT,42000.0,NY,115xx,USA,B,B3,Verified,221202.0,Individual,null,null,2024-03-12 17:34:...
fe2b4be6340070c54...,null,null,RENT,31000.0,NY,142xx,USA,D,D3,Source Verified,19100.0,Individual,null,null,2024-03-12 17:34:...
4de2852d38c8cbef6...,lead customer ser...,8 years,RENT,74000.0,AK,995xx,USA,B,B1,Not Verified,37000.0,Individual,null,null,2024-03-12 17:34:...
e573ae83004f28b50...,GM,4 years,OWN,38000.0,SC,295xx,USA,D,D4,Verified,23694.0,Individual,null,null,2024-03-12 17:34:...
f3aa6f48a5efd7aac...,Media Specialist,10+ years,MORTGAGE,60000.0,FL,327xx,USA,C,C5,Source Verified,3100.0,Individual,null,null,2024-03-12 17:34:...
307679b4ae65a4f28...,Mechanic,8 years,MORTGAGE,110000.0,PA,179xx,USA,C,C1,Not Verified,288400.0,Individual,null,null,2024-03-12 17:34:...
50fdd1738748332d6...,Respiratory care ...,2 years,OWN,60000.0,OK,731xx,USA,D,D5,Verified,173217.0,Individual,null,null,2024-03-12 17:34:...
f0349cb67df6ada13...,JOURNALIST,3 years,RENT,82000.0,DC,200xx,USA,B,B1,Source Verified,39300.0,Individual,null,null,2024-03-12 17:34:...
782d817a4150de56a...,CORRECTION OFFICER,10+ years,MORTGAGE,50000.0,OH,433xx,USA,C,C4,Source Verified,176971.0,Individual,null,null,2024-03-12 17:34:...


In [133]:
spark.sql("select count(*) from customers where annual_income is null")

count(1)
5


In [134]:
customers_income_filtered = spark.sql("select * from customers where annual_income is not null")

In [135]:
customers_income_filtered.createOrReplaceTempView("customers")

In [136]:
from pyspark.sql.functions import regexp_replace, col

In [137]:
customers_employment_length_cleaned = customers_income_filtered.withColumn("emp_length",regexp_replace(col("emp_length"),"\D",""))

In [138]:
customers_employment_length_cleaned

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
53cb0bbcffa80ba4a...,Nurse assistant,10,RENT,52000.0,CA,944xx,USA,C,C3,Not Verified,48435.0,Individual,null,null,2024-03-12 17:35:...
a458913dcc4537324...,maintenance,10,MORTGAGE,105000.0,IN,473xx,USA,B,B3,Not Verified,243004.0,Individual,null,null,2024-03-12 17:35:...
30570c6690c9ac315...,null,null,OWN,30000.0,NC,275xx,USA,A,A5,Not Verified,53200.0,Individual,null,null,2024-03-12 17:35:...
939beeb844750e341...,Sr IT Business An...,10,MORTGAGE,90000.0,NC,280xx,USA,A,A5,Verified,63605.0,Individual,null,null,2024-03-12 17:35:...
52cc002a63722b3a6...,K3 Life Skills Te...,3,RENT,33000.0,TN,371xx,USA,C,C1,Source Verified,41000.0,Individual,null,null,2024-03-12 17:35:...
8b70dd165df74149f...,Vice President,10,MORTGAGE,160000.0,OK,740xx,USA,A,A4,Not Verified,575280.0,Joint App,188000.0,Not Verified,2024-03-12 17:35:...
3a4139206f262a884...,irrigation tec,10,MORTGAGE,48000.0,MI,480xx,USA,B,B1,Not Verified,41832.0,Individual,null,null,2024-03-12 17:35:...
837809100b1bac449...,Mechanic,10,RENT,59000.0,TX,761xx,USA,A,A3,Verified,81900.0,Individual,null,null,2024-03-12 17:35:...
d2e6f70a74c915f88...,Operations Manager,9,OWN,125000.0,NY,112xx,USA,B,B3,Not Verified,524736.0,Individual,null,null,2024-03-12 17:35:...
11fe2f6f7027fe092...,Sr Administrative...,3,RENT,79800.0,CA,940xx,USA,B,B2,Source Verified,97037.0,Joint App,132800.0,Not Verified,2024-03-12 17:35:...


In [139]:
customers_employment_length_cleaned.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- address_zipcode: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- total_high_credit_limit: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [140]:
customers_emplength_casted = customers_employment_length_cleaned.withColumn("emp_length",customers_employment_length_cleaned.emp_length.cast('int'))

In [141]:
customers_emplength_casted

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
dd487e4d363d7f6d3...,assist ink blender,10,OWN,55000.0,WI,549xx,USA,B,B1,Not Verified,119011.0,Individual,null,null,2024-03-12 17:35:...
66c0acba3f4565c1b...,General Manager,10,MORTGAGE,225000.0,VT,053xx,USA,B,B1,Source Verified,119697.0,Individual,null,null,2024-03-12 17:35:...
ba3a38adb6cb9fcf8...,PASS Program Manager,1,MORTGAGE,92000.0,IL,606xx,USA,C,C4,Verified,202769.0,Individual,null,null,2024-03-12 17:35:...
dba794120c4ead047...,Court Officer,10,MORTGAGE,75000.0,NY,125xx,USA,C,C4,Not Verified,297921.0,Individual,null,null,2024-03-12 17:35:...
1d2923d0303f641c0...,Computer Specialist,5,OWN,49000.0,NY,113xx,USA,B,B5,Source Verified,91725.0,Individual,null,null,2024-03-12 17:35:...
30bf0014a37a26b6b...,shareholder,10,OWN,190000.0,NJ,070xx,USA,C,C3,Not Verified,793541.0,Individual,null,null,2024-03-12 17:35:...
dfc18dc83aebfedc9...,Admin. Asst./HR,10,OWN,36000.0,AL,361xx,USA,D,D2,Not Verified,27400.0,Individual,null,null,2024-03-12 17:35:...
2c3cf0c7a084104a2...,Support Engineer,2,MORTGAGE,71000.0,AL,350xx,USA,C,C4,Not Verified,187187.0,Individual,null,null,2024-03-12 17:35:...
eb1eda3afc50c1293...,owner,10,OWN,36000.0,FL,337xx,USA,C,C3,Source Verified,53394.0,Individual,null,null,2024-03-12 17:35:...
75a4866b295496320...,CEO,4,MORTGAGE,80000.0,FL,330xx,USA,B,B1,Not Verified,49599.0,Individual,null,null,2024-03-12 17:35:...


In [142]:
customers_emplength_casted.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- address_state: string (nullable = true)
 |-- address_zipcode: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- total_high_credit_limit: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- join_annual_income: float (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [143]:
customers_emplength_casted.filter("emp_length is null").count()

146903

In [144]:
customers_emplength_casted.createOrReplaceTempView("customers")

In [145]:
avg_emp_length = spark.sql("select floor(avg(emp_length)) as avg_emp_length from customers").collect()

In [146]:
print(avg_emp_length)

[Row(avg_emp_length=6)]


In [147]:
avg_emp_duration= avg_emp_length[0][0]

In [148]:
avg_emp_duration

6

In [149]:
customers_emplength_replaced= customers_emplength_casted.na.fill(avg_emp_duration,subset= ['emp_length'])

In [150]:
customers_emplength_replaced

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
9c81900ff2ec02898...,Paralegal,1,RENT,48006.0,MO,630xx,USA,D,D2,Not Verified,76250.0,Individual,null,null,2024-03-12 17:36:...
80eb97c8525c521a0...,sole proprietor,10,MORTGAGE,150000.0,CO,801xx,USA,C,C1,Source Verified,687833.0,Individual,null,null,2024-03-12 17:36:...
987f83e7ddeee7365...,Educator,10,MORTGAGE,98000.0,CA,934xx,USA,A,A3,Not Verified,55500.0,Individual,null,null,2024-03-12 17:36:...
45323dee25640adf6...,Planner,3,MORTGAGE,60000.0,CA,917xx,USA,A,A3,Verified,556208.0,Joint App,127000.0,Verified,2024-03-12 17:36:...
534c96bb347cab971...,Managing member,5,MORTGAGE,150000.0,CA,908xx,USA,A,A5,Source Verified,569147.0,Individual,null,null,2024-03-12 17:36:...
8de294e3b31ba4180...,Tax Program Assis...,10,RENT,49795.0,CA,958xx,USA,D,D2,Not Verified,29600.0,Individual,null,null,2024-03-12 17:36:...
00354b439913ee7c6...,null,6,MORTGAGE,19200.0,CA,923xx,USA,B,B3,Verified,188596.0,Joint App,94200.0,Verified,2024-03-12 17:36:...
562f74ab41124b2a5...,Polisher,4,RENT,50453.0,RI,029xx,USA,C,C4,Source Verified,30147.0,Individual,null,null,2024-03-12 17:36:...
b3a007ba42e4efccf...,null,6,MORTGAGE,40000.0,CA,959xx,USA,C,C3,Verified,193794.0,Individual,null,null,2024-03-12 17:36:...
71ac5968b03476e09...,Realtor,1,RENT,55000.0,NV,891xx,USA,C,C3,Source Verified,54500.0,Individual,null,null,2024-03-12 17:36:...


In [151]:
customers_emplength_replaced.filter('emp_length is null').count()

0

In [152]:
customers_emplength_replaced.createOrReplaceTempView("customers")

In [153]:
spark.sql("select distinct(address_state) from customers")

address_state
Helping Kenya's D...
175 (total projec...
223xx
SC
AZ
"so Plan """"C"""" is ..."
I am 56 yrs. old ...
financially I mad...
but no one will l...
LA


In [154]:
spark.sql("select count(address_state) from customers where length(address_state)>2")

count(address_state)
254


In [155]:
from pyspark.sql.functions import when,col,length

In [156]:
customers_state_cleaned = customers_emplength_replaced.withColumn(
    "address_state", 
    when(length(col("address_state"))>2,"NA").otherwise(col("address_state"))
                                        )

In [157]:
customers_state_cleaned

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
bc7fca8290c1e4aa7...,Beal Service Corp,6,MORTGAGE,84000.0,TX,750xx,USA,C,C2,Not Verified,null,Individual,null,null,2024-03-12 17:37:...
f6c4f46c27a75c915...,Retired From Texa...,10,RENT,39000.0,TX,770xx,USA,F,F1,Verified,null,Individual,null,null,2024-03-12 17:37:...
4c704c28ec2315673...,DHL Express,2,RENT,55000.0,NY,104xx,USA,E,E4,Not Verified,null,Individual,null,null,2024-03-12 17:37:...
aa8ff6e88731f81f2...,"Rock, Paper, Scis...",1,MORTGAGE,20000.0,GA,300xx,USA,F,F4,Not Verified,null,Individual,null,null,2024-03-12 17:37:...
c3bea8c7053491dff...,null,6,OWN,58000.0,NC,281xx,USA,C,C4,Verified,66400.0,Individual,null,null,2024-03-12 17:37:...
0fd88be5056f1af5c...,USAF,7,RENT,70000.0,CA,934xx,USA,C,C5,Verified,95950.0,Individual,null,null,2024-03-12 17:37:...
da15b90d4a557c56f...,Warehouse manager,5,RENT,54000.0,NJ,073xx,USA,B,B2,Not Verified,34224.0,Individual,null,null,2024-03-12 17:37:...
aa354703c3b80495b...,Owner,2,RENT,50000.0,MA,021xx,USA,D,D2,Source Verified,24700.0,Individual,null,null,2024-03-12 17:37:...
19a59b8ce40b875d4...,CEO,9,MORTGAGE,72000.0,CA,907xx,USA,F,F2,Not Verified,421017.0,Individual,null,null,2024-03-12 17:37:...
110d8654b3a30eb1e...,Manager/Funeral D...,5,RENT,70600.0,OK,740xx,USA,B,B4,Source Verified,50565.0,Individual,null,null,2024-03-12 17:37:...


In [158]:
customers_state_cleaned.select("address_state").distinct()

address_state
AZ
SC
LA
MN
NJ
DC
OR
NA
VA
null


In [159]:
customers_state_cleaned.write \
.format("parquet") \
.mode("overwrite") \
.option("path","/user/itv009959/lendingclubproject/cleaned/customers_parquet") \
.save()

In [160]:
customers_state_cleaned.write \
.option("header","true") \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv009959/lendingclubproject/cleaned/customers_csv") \
.save()

In [161]:
spark.stop()